In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/coinbaseBTCUSD_1min_2014-12-01_to_2017-10-20.csv")
df.timestamp = pd.to_datetime(df.timestamp, unit='s') # timestamp is in seconds
df.index = df.timestamp
del df['timestamp']
df = df.loc['2014-12-01T06:00:00':'2017-10-19T23:59:00'] # remove rows that do no lie within the hour window

In [5]:
df.head(1)

,open,high,low,close,volbtc,volusd,adjprice
timestamp,,,,,,,
2014-12-01 06:00:00,300.0,300.0,300.0,300.0,0.01,3.0,300.0


In [6]:
df.tail(1)

,open,high,low,close,volbtc,volusd,adjprice
timestamp,,,,,,,
2017-10-19 23:59:00,5703.96,5704.01,5702.87,5704.01,2.489754,14200.042922,5703.392752


In [9]:
# upsample to hourly
hr_df = df.resample(rule = "60Min").agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 
                                        'volbtc':'sum', 'volusd':'sum', 'adjprice':'last'})

In [10]:
hr_df.tail()

,open,high,low,close,volbtc,volusd,adjprice
timestamp,,,,,,,
2017-10-19 19:00:00,5673.11,5673.11,5626.00,5634.96,468.387320,2.648070e+06,5634.950625
2017-10-19 20:00:00,5634.96,5719.49,5634.95,5701.69,557.529716,3.164202e+06,5701.690000
2017-10-19 21:00:00,5701.69,5719.71,5691.09,5719.70,399.429587,2.279203e+06,5719.697723
2017-10-19 22:00:00,5719.70,5730.00,5690.00,5711.71,421.897866,2.412848e+06,5707.476979
2017-10-19 23:00:00,5711.71,5724.99,5689.97,5704.01,277.379961,1.584357e+06,5703.392752


In [11]:
hr_df.describe()

,open,high,low,close,volbtc,volusd,adjprice
count,24325.000000,24325.000000,24325.000000,24325.000000,24325.000000,2.432500e+04,24325.000000
mean,972.279134,976.970376,966.768851,972.481490,373.079144,4.659577e+05,972.456965
std,1131.419776,1138.376327,1123.862281,1131.811887,550.600701,1.016997e+06,1131.756870
min,114.030000,150.000000,0.060000,113.060000,0.001109,4.188129e+00,139.895000
25%,280.100000,281.000000,279.360000,280.090000,164.423958,6.938169e+04,280.108260
50%,532.460000,534.160000,528.720000,532.550000,265.881857,1.256844e+05,532.535696
75%,1011.810000,1013.940000,1008.000000,1011.820000,434.613093,3.317424e+05,1011.780037
max,5849.100000,5867.000000,5820.190000,5849.100000,31505.461253,2.856359e+07,5849.097036
